## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Kirakira,-10.4544,161.9205,83.26,78,100,7.40,SB,2021-12-06 22:34:24,overcast clouds
1,1,Albany,42.6001,-73.9662,58.08,88,100,11.54,US,2021-12-06 22:30:48,light rain
2,2,Yellowknife,62.4560,-114.3525,-16.58,83,75,4.61,CA,2021-12-06 22:31:14,broken clouds
3,3,Roald,62.5833,6.1333,37.09,59,75,20.71,NO,2021-12-06 22:38:43,broken clouds
4,4,Derzhavinsk,51.1000,66.3167,20.82,97,100,5.28,KZ,2021-12-06 22:38:44,light snow


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Kirakira,-10.4544,161.9205,83.26,78,100,7.40,SB,2021-12-06 22:34:24,overcast clouds
6,6,Rikitea,-23.1203,-134.9692,76.42,72,73,8.50,PF,2021-12-06 22:32:14,broken clouds
7,7,Quepos,9.4236,-84.1652,87.10,86,77,2.37,CR,2021-12-06 22:38:44,broken clouds
16,16,Puerto Ayora,-0.7393,-90.3518,76.96,78,54,9.10,EC,2021-12-06 22:32:33,broken clouds
24,24,Pemangkat,1.1667,108.9667,79.43,83,100,6.91,ID,2021-12-06 22:34:43,overcast clouds
26,26,Hilo,19.7297,-155.0900,75.25,92,90,6.91,US,2021-12-06 22:32:27,heavy intensity rain
28,28,Atuona,-9.8000,-139.0333,79.12,67,11,19.53,PF,2021-12-06 22:32:44,few clouds
29,29,Pedernales,18.0384,-71.7440,80.53,67,0,8.21,DO,2021-12-06 22:38:48,clear sky
31,31,Harper,4.3750,-7.7169,79.27,84,56,9.93,LR,2021-12-06 22:33:46,broken clouds
34,34,Vaini,-21.2000,-175.2000,82.56,83,75,10.36,TO,2021-12-06 22:32:15,broken clouds


In [13]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull().sum()


City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        1
Date           0
Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [16]:
clean_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        0
Date           0
Description    0
dtype: int64

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Kirakira,SB,83.26,overcast clouds,-10.4544,161.9205,
6,Rikitea,PF,76.42,broken clouds,-23.1203,-134.9692,
7,Quepos,CR,87.10,broken clouds,9.4236,-84.1652,
16,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
24,Pemangkat,ID,79.43,overcast clouds,1.1667,108.9667,
26,Hilo,US,75.25,heavy intensity rain,19.7297,-155.0900,
28,Atuona,PF,79.12,few clouds,-9.8000,-139.0333,
29,Pedernales,DO,80.53,clear sky,18.0384,-71.7440,
31,Harper,LR,79.27,broken clouds,4.3750,-7.7169,
34,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Conditions</dt><dd>{Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))